# Emerging Technolgoies

### Libraries 

In [3]:
# Random selections.
import random

# Numerical arrays.
import numpy as np

## Problem 1: Generating Random Boolean Functions


In [4]:
# Constant function 1.
# Constant functions do not depend on any input
# they always return the same value (0 or 1), regardless of
# the values of their arguments.
def f1(a,b,c,d):
  """Takes a single bit but always returns 0."""
  return 0

# Balanced function 1.
def f2(a,b,c,d):
  """Identity function."""
  return a ^ b

# Balanced function 2.
def f3(a,b,c,d):
  """Not function."""
  return a ^ b ^ c ^ d

# constant function 2. 
def f4(a,b,c,d):
  """Constant 1 function."""
  return 1

In [8]:
def random_constant_balanced():
  """Randomly returns one of the four functions defined above."""
  # List of potential functions.
  functions = [f1, f2, f3, f4]
  # Return element at random.
  random_function = random.choice(functions)
  return random_function

random_constant_balanced()

<function __main__.f2(a, b, c, d)>

## Problem 2: Classical Testing for Function Type


## Problem 3: Quantum Oracles


## Problem 4: Deutsch's Algorithm with Qiskit


## Problem 5: Scaling to the Deutsch–Jozsa Algorithm
